In [4]:
import matplotlib.pyplot as plt
import calplot
import calmap
import csv
import sklearn
import pickle
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,train_test_split,StratifiedKFold,cross_val_score,learning_curve
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import os
from nltk.corpus import words 
from IPython.display import HTML, display
from datasketch import MinHash, MinHashLSH
import pandas as pd
from sklearn.model_selection import train_test_split
from datasketch import MinHash, MinHashLSH

<h2>Overview of the data and data source.</h2> 

This dataset contains and excert of Spam and Ham E-Mails from the Enron Corpus. 

Enron was an american company, that due to financial fraud collapse and subsequentiall was under investigation by law enforcement agencies. During these investigations, E-Mails servers were seized and later published. 

Today this corpus is special, as it is only one of the few datsets, that actually contain 'real' E-Mails ready for analysis. 

The entire Corpus is around 600000 E-Mails, for this project a smaller subset is being used for Spam detection. 

<h2>Goals of the project</h2>

The goal of the project is to analyze different Spam. Several goals: 

1. Find the best Spam filter
2. Determine which effect on the Spam recognition the Subject line has. 
3. Determine the best Dataset for Spam detection out of a number of datasets --> e.g. train with 5 datasets and test on 6 dataset

<h2>Preprocessing and Cleaning</h2<

In [5]:
data_file = './enron_spam_data.csv'

In [6]:
enron_df = pd.read_csv(data_file)

In [7]:
enron_df_original = pd.read_csv(data_file)

In [8]:
header_names = enron_df.keys()

In [9]:
print(header_names)

Index(['Message ID', 'Subject', 'Message', 'Spam/Ham', 'Date'], dtype='object')


In [10]:
enron_df.head()

,Message ID,Subject,Message,Spam/Ham,Date
0,0,christmas tree farm pictures,NaN,ham,1999-12-10
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14


In [11]:
enron_df.describe()

,Message ID
count,33716.000000
mean,16857.500000
std,9733.115174
min,0.000000
25%,8428.750000
50%,16857.500000
75%,25286.250000
max,33715.000000


In [12]:
enron_label = enron_df['Spam/Ham'].value_counts()
print(enron_label)

spam    17171
ham     16545
Name: Spam/Ham, dtype: int64


In [13]:
enron_df['Spam/Ham'].value_counts(normalize=True)

spam    0.509283
ham     0.490717
Name: Spam/Ham, dtype: float64

Noticing the NaN in the first message shown above, I decided to anayze how complete the dataset is. 

In [14]:
messageID_NaN = enron_df['Message ID'].isna().sum()
subject_NaN = enron_df['Subject'].isna().sum()
message_NaN = enron_df['Message'].isna().sum()
spam_NaN = enron_df['Spam/Ham'].isna().sum()
print(f"Number of NaN values in 'Message ID' column: {messageID_NaN}")
print(f"Number of NaN values in 'Subject' column: {subject_NaN}")
print(f"Number of NaN values in 'Message' column: {message_NaN}")
print(f"Number of NaN values in 'Spam/Ham' column: {spam_NaN}")

Number of NaN values in 'Message ID' column: 0
Number of NaN values in 'Subject' column: 289
Number of NaN values in 'Message' column: 371
Number of NaN values in 'Spam/Ham' column: 0


In [15]:
max_nan = subject_NaN + message_NaN
percentage_nan = max_nan / len(enron_df)
print('Total Maximum Lost E-Mails:' ,max_nan)
print('Percentage of Total E-Mails:',percentage_nan)

Total Maximum Lost E-Mails: 660
Percentage of Total E-Mails: 0.019575275833432198


As we can see, a maximum of 660 E-Mails would be lost if we dropped all NaN values. While this is only 1,9% of all E-Mails, we risk eleminating Spam messages and tainting the analysis. Therefore I analyze how many of the 'subjectless' E-Mails are Spam.

In [16]:
na_and_spam_sub = len(enron_df[(enron_df["Subject"].isna()) & (enron_df["Spam/Ham"]=='spam')])
na_and_spam_mes = len(enron_df[(enron_df["Message"].isna()) & (enron_df["Spam/Ham"]=='spam')])
print('Subject = NaN && is Spam:',na_and_spam_sub)
print('Message = NaN && is spam:' ,na_and_spam_mes)

Subject = NaN && is Spam: 289
Message = NaN && is spam: 319


Interestingly enough, all of the message where there is no subject are classified as Spam.
For Messages with no Body, this happens also for most of the messages

In [17]:
print(enron_df[enron_df['Subject'].isna()])

       Message ID Subject                                            Message  \
3699         3699     NaN  holiday specials on : didrex and nasacort\n60 ...   
3708         3708     NaN  the only solution to penis\nenlargement\nnzzbq...   
3749         3749     NaN                                                NaN   
3777         3777     NaN  to _ cc _ default _ handler\nsubject : [ spam ...   
3820         3820     NaN  discount meds right from home\nvalium , xanax ...   
...           ...     ...                                                ...   
32833       32833     NaN  why paily for pobrne ?\nas a lrpim " itecd ti ...   
32835       32835     NaN  are youu stil 3 l pa margin - right : 8 " alig...   
32898       32898     NaN  pre - ejeculators :\nplease read here for info...   
32926       32926     NaN  there ' s never been a better time to re _ fin...   
33155       33155     NaN  sud euler gangling phoneme billy birdlike dutc...   

      Spam/Ham        Date  
3699      

Taking a quick peek at these couple of message, they indeed appear to be Spam. 

Let us now count the number of message where there is no subject line and no body: 

In [18]:
empty_message = len(enron_df[(enron_df["Subject"].isna()) & (enron_df["Message"].isna())])

In [19]:
print(empty_message)

51


In [20]:
print(enron_df[(enron_df["Subject"].isna()) & (enron_df["Message"].isna())])

       Message ID Subject Message Spam/Ham        Date
3749         3749     NaN     NaN     spam  2004-01-24
4062         4062     NaN     NaN     spam  2004-06-22
4136         4136     NaN     NaN     spam  2004-07-22
4141         4141     NaN     NaN     spam  2004-07-24
4153         4153     NaN     NaN     spam  2004-07-29
4254         4254     NaN     NaN     spam  2004-09-04
4282         4282     NaN     NaN     spam  2004-09-15
4593         4593     NaN     NaN     spam  2004-12-21
4596         4596     NaN     NaN     spam  2004-12-23
4601         4601     NaN     NaN     spam  2004-12-25
4683         4683     NaN     NaN     spam  2005-01-25
4684         4684     NaN     NaN     spam  2005-01-25
4691         4691     NaN     NaN     spam  2005-01-27
4706         4706     NaN     NaN     spam  2005-02-02
4728         4728     NaN     NaN     spam  2005-02-07
4963         4963     NaN     NaN     spam  2005-05-13
18420       18420     NaN     NaN     spam  2004-02-28
18581     

Indeed there are 51 message that do not contain Subject and Message. We will leave those in the dataframe, as they are nevertheless Spam. While they are probably not Harmful (maybe except for DoS against the server), they can be annoying for the enduse. 

As can be seen above, we notice the Column Date. While it is surely interesting to correlate the impending doom of financial collapse of the company to the amount of E-Mails send in that time frame, having the date for detecting Spams is not necessary. 

Potentially for later graphics it would be interesting to see, if Spam is sent on the weekend or not.

In any case, for now we can safely ignore the Date column: 

In [21]:
enron_df.drop('Date', axis=1, inplace = True)

In [22]:
df2 = enron_df[enron_df['Message'].duplicated() == True].sort_values('Message')

In [23]:
print(df2)

       Message ID                                            Subject  \
25239       25239                             how to soak her in cum   
25726       25726                             how to soak her in cum   
32524       32524  aggressive investors and traders shouid be wat...   
32508       32508      owning these shares fast - tracks performance   
32591       32591  this performer defines the secret of stock sel...   
...           ...                                                ...   
27999       27999                           ripley house photo album   
28722       28722  http : / / hrweb . enron . com / benefits / fo...   
29059       29059                          h : eyeforenergy briefing   
29100       29100                          h : eyeforenergy briefing   
29109       29109                                            website   

                                                 Message Spam/Ham  
25239  " i just wanted to write and thank you for spu...     spam  

Obviously if we consider the nature of Spam messages, it becomes clear that some of the received messages are Duplicates. I have decided to remove all of these duplicates, as we do not want to have any bias in the following analysis. 

In [24]:
enron_df.describe()

,Message ID
count,33716.000000
mean,16857.500000
std,9733.115174
min,0.000000
25%,8428.750000
50%,16857.500000
75%,25286.250000
max,33715.000000


In [25]:
enron_df.drop_duplicates(subset=['Message'], inplace = True)

In [26]:
enron_df.head()

,Message ID,Subject,Message,Spam/Ham
0,0,christmas tree farm pictures,NaN,ham
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham


In [27]:
enron_df.describe()

,Message ID
count,29780.000000
mean,16090.690262
std,9683.706508
min,0.000000
25%,7782.750000
50%,15597.500000
75%,23917.250000
max,33714.000000


In [28]:
df2 = len(enron_df[enron_df['Subject'].duplicated() == True])

In [29]:
print(df2)

6205


As we can see, there are also 6205 dubplicated subjects. This surprised me, as it represent a significant part of all the messages. NEverthless I decided to not remove those messages, as we have already remove messages.

As Python is casesensitive I have decided to transform all words into lowercase. ALthough it appears the dataset is already completly in lowercase, I do this for safety reasons.

In [30]:
enron_df['Message'] = enron_df['Message'].str.lower()
enron_df['Subject'] = enron_df['Subject'].str.lower()

In [31]:
enron_df.head()

,Message ID,Subject,Message,Spam/Ham
0,0,christmas tree farm pictures,NaN,ham
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham


In [32]:
enron_df["Message"] = enron_df['Message'].str.replace('[^\w\s]','', regex=True)
enron_df["Subject"] = enron_df['Subject'].str.replace('[^\w\s]','', regex=True)

In [33]:
enron_df.head()

,Message ID,Subject,Message,Spam/Ham
0,0,christmas tree farm pictures,NaN,ham
1,1,vastar resources inc,gary production from the high island larger b...,ham
2,2,calpine daily gas nomination,calpine daily gas nomination 1 doc,ham
3,3,re issue,fyi see note below already done \nstella\n ...,ham
4,4,meter 7268 nov allocation,fyi \n forwarded by lauri...,ham


In [34]:
stop = stopwords.words('english')
enron_df['Message'] = enron_df['Message'].fillna('')
enron_df['Subject'] = enron_df['Subject'].fillna('')

In [35]:
enron_df['Message'] = enron_df['Message'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
enron_df['Subject'] = enron_df['Subject'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [36]:
enron_df.head()

,Message ID,Subject,Message,Spam/Ham
0,0,christmas tree farm pictures,,ham
1,1,vastar resources inc,gary production high island larger block 1 2 c...,ham
2,2,calpine daily gas nomination,calpine daily gas nomination 1 doc,ham
3,3,issue,fyi see note already done stella forwarded ste...,ham
4,4,meter 7268 nov allocation,fyi forwarded lauri allen hou ect 12 14 99 12 ...,ham


As we can see removing punctuation and stopwords works fine. Sadly as we can see, due to reasons the NaN values had to be replaced with NaN. #################Potentially fix this alter

In the next step we divide the dataset into training data and testing data

Having thought about how to handle the subject line of the message, I decideded to view the subject line simply as a "Headline", and therefore decided to unfiy message and subject into one column. In a sense, this also solves our NaN problem, as it is now ensure that every anaylze E-Mail has a unified message we can look at. 

In [37]:
enron_df['unified'] = enron_df['Subject'] + ' ' + enron_df['Message']
enron_df_unified = enron_df.drop(['Subject', 'Message'], axis = 1)

In [38]:
enron_df.head()

,Message ID,Subject,Message,Spam/Ham,unified
0,0,christmas tree farm pictures,,ham,christmas tree farm pictures
1,1,vastar resources inc,gary production high island larger block 1 2 c...,ham,vastar resources inc gary production high isla...
2,2,calpine daily gas nomination,calpine daily gas nomination 1 doc,ham,calpine daily gas nomination calpine daily gas...
3,3,issue,fyi see note already done stella forwarded ste...,ham,issue fyi see note already done stella forward...
4,4,meter 7268 nov allocation,fyi forwarded lauri allen hou ect 12 14 99 12 ...,ham,meter 7268 nov allocation fyi forwarded lauri ...


In [39]:
enron_df_unified.head()

,Message ID,Spam/Ham,unified
0,0,ham,christmas tree farm pictures
1,1,ham,vastar resources inc gary production high isla...
2,2,ham,calpine daily gas nomination calpine daily gas...
3,3,ham,issue fyi see note already done stella forward...
4,4,ham,meter 7268 nov allocation fyi forwarded lauri ...


In [40]:
na_rows = enron_df_unified[enron_df_unified['Spam/Ham'].isna() | (enron_df_unified['Spam/Ham'] == ' ')]
print(na_rows)

Empty DataFrame
Columns: [Message ID, Spam/Ham, unified]
Index: []


As a last step, I decide to randomlay arrange the order of rows, so there is no bias: 

In [41]:
enron_df_unified = enron_df_unified.sample(frac=1)

In [42]:
enron_df_nb = enron_df_unified.copy()
enron_df_bl = enron_df_unified.copy()
enron_df_lsh = enron_df_unified.copy()

At this point, our dataframe is ready to be analyzed. We can begin with the training using the enron_df_uniied df. 

<h2>Naive Bayes Filter</h2>

We will start with the Naive Bayes Filter. In this stage we start to analyze firstly our unified version of the subject and the E-Mail body. 

Later we take a look how well this Bayes Filter works, when we analyze the Subject Line and the E-Mail body separatly.

Lastly I use a different way to implement the Bayes Filter that I found online: the description can be found here: https://www.kdnuggets.com/2020/07/spam-filter-python-naive-bayes-scratch.html

In [43]:
training_ratio = 0.7

<h3>Unified Subject Line and E-Mail Body</h3>

In [44]:
X_unified = enron_df_nb['unified']
y_unified = enron_df_nb['Spam/Ham']
X_train, X_test, y_train, y_test, idx_train, idx_test = \
    train_test_split(X_unified, y_unified, range(len(y_unified)), 
    train_size=training_ratio, random_state=1)

vectorizer = CountVectorizer()
X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

mnb = MultinomialNB()
mnb.fit(X_train_vector, y_train)
y_pred = mnb.predict(X_test_vector)

results_df = pd.concat([y_test.reset_index(drop=True), pd.Series(y_pred)], axis=1, keys=['true', 'predicted'])

print(classification_report(y_test, y_pred, target_names=['Spam', 'Ham']))
print('Classification accuracy {:.1%}'.format(accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

        Spam       0.99      0.99      0.99      4708
         Ham       0.99      0.98      0.99      4226

    accuracy                           0.99      8934
   macro avg       0.99      0.99      0.99      8934
weighted avg       0.99      0.99      0.99      8934

Classification accuracy 98.7%


In [45]:
wrong_predictions = 0
for i, row in results_df.iterrows():
    if row['true'] != row['predicted']:
        print(f"Predicted: {row['predicted']}, True: {row['true']},\nText: {X_test.iloc[i]}\n")
        wrong_predictions += 1

Predicted: spam, True: ham,
Text: boat believe boat 18 19 ft long boat cover bring boat information tomorrow anymore questions recently took shop get ready season salesman said looked brand new brand new battery everything runs perfectly call want take look briant 66459

Predicted: ham, True: spam,
Text: join focus groups earn money la carte research recruits focus groups across country focus groups easy way make extra money couple hours time group purpose learning opinions assured sales presentation asked buy anything everything mentioned held strictest confidence focus groups let express opinions whatever subject discussed actually pay opinions would like added list possible future respondents click fill registration form questions questionnaire please e mail register alacarteresearch com sincerely john mooney

Predicted: ham, True: spam,
Text: information closing notification made effort write many occasions last contact present loan situation meets requirements 3 70 lower rate howe

Predicted: spam, True: ham,
Text: address recommendations vince please forward recommendations following address 2569 windbreak dr alexandria va 22306 forget label envelope e g columbia etc sign envelope adhesive seal must receive package new year order meet submission deadline thank time happy holidays hector

Predicted: ham, True: spam,
Text: information final notification aimed make contact lot periods hope reply time current home loan makes eligible 3 70 lower rate however thanks previous attempts make contact succeed last effort get lower rate please end final step upon receiving notice immediately complete submission apply decision make use final offer going help

Predicted: ham, True: spam,
Text: nymex invitation learn power trading power trading fundamentals sept 15 16 nymex nyc early bird discount effect nymex power delegates learn electricity markets overview simulated trading exercise market factors basic trading tools new york mercantile exchange financial instruments optio

In [46]:
print(f"Number of wrong predictions: {wrong_predictions}")

Number of wrong predictions: 119


<h1>#########NOTE: NEED TO ANALYZE THIS NOW</h1>

<h3>Analyzing only the subject line</h3>

Let us now try to analyze what happens if we only observe the Subject headers. How does this affect the Spam detection accuracy?

My suspicion is, that this will lower the recognition rate of Spam significantly. I even suspect it will make the Spam Filter completly worthless, with possible prediction rates approach 60% or less.

In [47]:
subject_NaN = enron_df['Subject'].isna().sum()
subject_empty = (enron_df['Subject'] == ' ').sum()

print(subject_NaN)
print(subject_empty)

0
0


In [48]:
X_subject = enron_df['Subject']
y_subject = enron_df['Spam/Ham']
X_train, X_test, y_train, y_test, idx_train, idx_test = \
    train_test_split(X_subject, y_subject, range(len(y_unified)), 
    train_size=training_ratio, random_state=1)

vectorizer = CountVectorizer()
X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

mnb = MultinomialNB()
mnb.fit(X_train_vector, y_train)
y_pred = mnb.predict(X_test_vector)

results_df = pd.concat([y_test.reset_index(drop=True), pd.Series(y_pred)], axis=1, keys=['true', 'predicted'])

print(classification_report(y_test, y_pred, target_names=['Spam', 'Ham']))
print('Classification accuracy {:.1%}'.format(accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

        Spam       0.88      0.95      0.91      4732
         Ham       0.94      0.85      0.89      4202

    accuracy                           0.90      8934
   macro avg       0.91      0.90      0.90      8934
weighted avg       0.91      0.90      0.90      8934

Classification accuracy 90.3%


As we can see, even when only anayzing the subject line it can still reach a correct predicition of 90.3%, which is not great but still much better than expected. Still quite useless though and it confirms the suspicion that only using the Subject Line to analyze Spam is definitly not enough.

In [49]:
wrong_predictions = 0
for i, row in results_df.iterrows():
    if row['true'] != row['predicted']:
        print(f"Predicted: {row['predicted']}, True: {row['true']},\nText: {X_test.iloc[i]}\n")
        wrong_predictions += 1

Predicted: ham, True: spam,
Text: 10

Predicted: ham, True: spam,
Text: findikkiran

Predicted: ham, True: spam,
Text: 1

Predicted: ham, True: spam,
Text: fw hotelman

Predicted: ham, True: spam,
Text: 

Predicted: spam, True: ham,
Text: replacing people left

Predicted: ham, True: spam,
Text: report ufhvv

Predicted: ham, True: spam,
Text: toda linha smilinguido

Predicted: ham, True: spam,
Text: urgentt ransaction

Predicted: spam, True: ham,
Text: kingwood cove 2 play 20

Predicted: spam, True: ham,
Text: last chance

Predicted: ham, True: spam,
Text: b

Predicted: spam, True: ham,
Text: last check

Predicted: ham, True: spam,
Text: smailcap journa presents

Predicted: ham, True: spam,
Text: george williams

Predicted: ham, True: spam,
Text: fw surprise

Predicted: ham, True: spam,
Text: xvuzkp

Predicted: ham, True: spam,
Text: 

Predicted: ham, True: spam,
Text: 

Predicted: ham, True: spam,
Text: finance understaning important

Predicted: spam, True: ham,
Text: gallup news

Pred

Predicted: ham, True: spam,
Text: hello

Predicted: ham, True: spam,
Text: safety issue x

Predicted: ham, True: spam,
Text: 9

Predicted: ham, True: spam,
Text: bike ride weekend

Predicted: ham, True: spam,
Text: letter intent

Predicted: ham, True: spam,
Text: deforest

Predicted: ham, True: spam,
Text: code xbl 28

Predicted: ham, True: spam,
Text: wakey wakey

Predicted: ham, True: spam,
Text: ceo executive management hr manager

Predicted: ham, True: spam,
Text: lagrangian

Predicted: ham, True: spam,
Text: creativity innovation newsletter march 2005 vol 1

Predicted: ham, True: spam,
Text: letter mary

Predicted: ham, True: spam,
Text: appointment wednesday 05 00

Predicted: spam, True: ham,
Text: hi

Predicted: ham, True: spam,
Text: attention director urgent contract supply

Predicted: spam, True: ham,
Text: help cluster analysis

Predicted: ham, True: spam,
Text: information update number 565

Predicted: ham, True: spam,
Text: good time party

Predicted: ham, True: spam,
Text

In [50]:
print(f"Number of wrong predictions: {wrong_predictions}")

Number of wrong predictions: 868


<h3>Analyzing only the E-Mail Body</h3>

Now lets try the same with only the message body and see how much effect the Subject has on the Spam detection. 

In [51]:
X_body = enron_df['Message']
y_body = enron_df['Spam/Ham']
X_train, X_test, y_train, y_test, idx_train, idx_test = \
    train_test_split(X_body, y_body, range(len(y_unified)), 
    train_size=training_ratio, random_state=1)

vectorizer = CountVectorizer()
X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

mnb = MultinomialNB()
mnb.fit(X_train_vector, y_train)
y_pred = mnb.predict(X_test_vector)

results_df = pd.concat([y_test.reset_index(drop=True), pd.Series(y_pred)], axis=1, keys=['true', 'predicted'])

print(classification_report(y_test, y_pred, target_names=['Spam', 'Ham']))
print('Classification accuracy {:.1%}'.format(accuracy_score(y_test, y_pred)))

              precision    recall  f1-score   support

        Spam       0.98      0.99      0.99      4732
         Ham       0.99      0.98      0.98      4202

    accuracy                           0.98      8934
   macro avg       0.98      0.98      0.98      8934
weighted avg       0.98      0.98      0.98      8934

Classification accuracy 98.4%


It has 98.4%! This is quite good. But still less than the unification of the body and the Subject. Therefore it makes sense to use the first method which unfies both Subject Line and E-Mail body. 

For future analysis and representation, only the first analysis type will be considered

<h2>############### TO DO Blacklisting --> Need to randomize order of files</h2>

In [52]:
X_train_bl = enron_df_bl[:int(len(enron_df_bl)*training_ratio)]
X_test_bl = enron_df_bl[int(len(enron_df_bl)*training_ratio):]
x_train_Spam_bl = enron_df_bl['Spam/Ham']
stemmer = nltk.PorterStemmer()
stopwords = set(nltk.corpus.stopwords.words('english'))
spam_words = set()
ham_words = set()


for _, row in X_train_bl.iterrows():
    path = row['unified']
    label = row['Spam/Ham']
    tokens = nltk.word_tokenize(path)
    if len(tokens) > 2:
        stems = [stemmer.stem(w) for w in tokens if w not in stopwords]
    if not stems:
        continue
    if label == 'ham':
        ham_words.update(stems)
    elif label == 'spam':
        spam_words.update(stems)
    else:
        continue

blacklist = spam_words - ham_words
pickle.dump(blacklist, open('blacklist.pkl', 'wb'))

print(len(spam_words))
print(len(ham_words))
print('Blacklist of {} tokens successfully built/loaded'.format(len(blacklist)))

87585
39030
Blacklist of 70206 tokens successfully built/loaded


In [53]:
word_set = set(words.words()) 
word_set.intersection(blacklist)

{'brocket',
 'verb',
 'knitwear',
 'coulomb',
 'shuttlecock',
 'confessor',
 'plantain',
 'becap',
 'hypothyroid',
 'rondo',
 'horsehead',
 'helmsman',
 'proconsul',
 'spasm',
 'gabbro',
 'ravenduck',
 'diphthong',
 'bestir',
 'thereto',
 'chalet',
 'fling',
 'nymphomaniac',
 'prover',
 'isopleth',
 'serviceman',
 'tailorless',
 'sparer',
 'striker',
 'ytterbium',
 'mila',
 'copra',
 'grandstand',
 'overthrow',
 'hookah',
 'strass',
 'thousandth',
 'spiro',
 'shriek',
 'larboard',
 'diopter',
 'deadbeat',
 'bra',
 'flection',
 'propupa',
 'dither',
 'avouch',
 'archfool',
 'sabbath',
 'trove',
 'undress',
 'transom',
 'demiss',
 'hairdress',
 'radix',
 'churchwoman',
 'radiogram',
 'dopper',
 'piso',
 'pincer',
 'comprador',
 'threader',
 'lamplight',
 'counterflow',
 'fop',
 'mausoleum',
 'hoydenish',
 'rhododendron',
 'quack',
 'hoodwink',
 'pornograph',
 'dieter',
 'socialist',
 'coquina',
 'rabbin',
 'tomato',
 'wailer',
 'cistern',
 'amra',
 'foal',
 'voluntary',
 'unbutton',
 'wu

In [54]:
fp = 0
tp = 0
fn = 0
tn = 0

for _, row in X_test_bl.iterrows():
    path = row['unified']
    label = row['Spam/Ham']
    tokens = nltk.word_tokenize(path)
    if len(tokens) > 2:
        stems = [stemmer.stem(w) for w in tokens if w not in stopwords]
    if not stems:
        continue
    stems_set = set(stems)
    if stems_set & blacklist: # email's words are in blacklist
        if label == 'ham': # ham
            fp = fp + 1 
        else:
            tp = tp + 1
    else: # email's words are not in blacklist
        if label == 'ham':
            tn = tn + 1
        else:
            fn = fn + 1

In [55]:
conf_matrix = [[tn, fp],
               [fn, tp]]
display(HTML('<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>{}</td>'.format(
        '</td><td>'.join(str(_) for _ in row)) 
                     for row in conf_matrix))))

4005,744
325,3860


In [56]:
count = tn + tp + fn + fp
percent_matrix = [["{:.1%}".format(tn/count), "{:.1%}".format(fp/count)],
                  ["{:.1%}".format(fn/count), "{:.1%}".format(tp/count)]]
display(HTML('<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>{}</td>'.format(
        '</td><td>'.join(str(_) for _ in row)) 
                     for row in percent_matrix))))

44.8%,8.3%
3.6%,43.2%


In [57]:
print("Classification accuracy: {}".format("{:.1%}".format((tp+tn)/count)))

Classification accuracy: 88.0%


<h2>LSH ####################### fix classification accuracy</h2>

In [174]:
X_train_lsh = enron_df_lsh[:int(len(enron_df_lsh)*training_ratio)]
X_test_lsh = enron_df_lsh[int(len(enron_df_lsh)*training_ratio):]
X_spam_files = X_train_lsh[X_train_lsh['Spam/Ham'] == 'spam']

In [175]:
X_spam_files.head()

,Message ID,Spam/Ham,unified
9945,9945,spam,fantastic investors info maisonette internatio...
24551,24551,spam,cost effective mlm bizop phone screened leads ...
4102,4102,spam,solution penis enlargement limited time offer...
31136,31136,spam,new discovery may help fight cancer ancient se...
22077,22077,spam,new wireless products wireless products 14 35 ...


In [176]:
lsh = MinHashLSH(threshold=0.5, num_perm=128)

In [177]:
for idx, row in X_spam_files.iterrows():
    minhash = MinHash(num_perm=128)
    path = row['unified']
    tokens = nltk.word_tokenize(path)
    stems = [stemmer.stem(w) for w in tokens if w not in stopwords]
    if len(stems) < 2: continue
    for s in stems:
        minhash.update(s.encode('utf-8'))
    key = str(row['Message ID'])
    lsh.insert(key, minhash)

In [178]:
def lsh_predict_label(stems):
    '''
    Queries the LSH matcher and returns:
        'spam' if predicted spam
        'ham' if predicted ham
       -1 if parsing error
    '''
    minhash = MinHash(num_perm=128)
    if len(stems) < 2:
        return -1
    for s in stems:
        minhash.update(s.encode('utf-8'))
    matches = lsh.query(minhash)
    if matches:
        return 'spam'
    else:
        return 'ham'

In [239]:
fp = 0
tp = 0
fn = 0
tn = 0
false_negatives = []
true_negatives = []
false_positives = []
true_positives = []

for f, row in X_test_lsh.iterrows():
    label = row['Spam/Ham']
    path = row['unified']
    message_id = row['Message ID']
    tokens = nltk.word_tokenize(path)
    if not tokens:
        continue
    stems = [stemmer.stem(w) for w in tokens if w not in stopwords]
    if not stems:
        continue
    pred = lsh_predict_label(stems)
    if pred == -1:
        continue
    elif pred == 'spam': # predicted spam
        if label == 'ham':
            fp = fp + 1
            false_positives.append((message_id, pred, label, path))
        else:
            tp = tp + 1
            true_positives.append((message_id, pred, label, path))
    elif pred == 'ham': # predicted ham
        if label == 'ham':
            tn = tn + 1
            if(len(true_negatives) < 50):
                true_negatives.append((message_id, pred, label, path))
        else:
            fn = fn + 1
            if(len(false_negatives) < 20):
                false_negatives.append((message_id, pred, label, path))

In [240]:
from IPython.display import HTML, display
conf_matrix = [[tn, fp],
               [fn, tp]]
display(HTML('<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>{}</td>'.format(
        '</td><td>'.join(str(_) for _ in row)) 
                     for row in conf_matrix))))

4684,65
2221,1963


In [241]:
count = tn + tp + fn + fp
percent_matrix = [["{:.1%}".format(tn/count), "{:.1%}".format(fp/count)],
                  ["{:.1%}".format(fn/count), "{:.1%}".format(tp/count)]]
display(HTML('<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>{}</td>'.format(
        '</td><td>'.join(str(_) for _ in row)) 
                     for row in percent_matrix))))

52.4%,0.7%
24.9%,22.0%


In [242]:
print("Classification accuracy: {}".format("{:.1%}".format((tp+tn)/count)))

Classification accuracy: 74.4%


In [254]:
msg_id = 13980

row = enron_df_original.loc[enron_df_original['Message ID'] == msg_id].iloc[0]

print("Message ID: ", row['Message ID'])
print("Subject: ", row['Subject'])
print("\n")
print(row['Message'])
print(row['Spam/Ham'])


Message ID:  13980
Subject:  trade counts through 8 : 30 am


gas down some today , but power up .
trade date na gas na power total trade cnt
11 / 16 / 2001 880 583 2203
11 / 15 / 2001 1006 523 2199
11 / 14 / 2001 731 444 2364
11 / 13 / 2001 868 527 1951
11 / 12 / 2001 878 380 1913
11 / 11 / 2001 7 2 9
11 / 10 / 2001 5 8 13
11 / 9 / 2001 615 444 1679
11 / 8 / 2001 975 489 1931
11 / 7 / 2001 877 727 2288
11 / 6 / 2001 1064 849 2463
11 / 5 / 2001 930 839 2308
11 / 4 / 2001 4 3 7
11 / 3 / 2001 2 16 18
11 / 2 / 2001 1211 576 2253
ham


In [255]:
print(X_test_lsh[X_test_lsh['Message ID'] == 13980])

       Message ID Spam/Ham                                            unified
13980       13980      ham  trade counts 8 30 gas today power trade date n...


In [256]:
print("True negatives:")
for msg_id, pred, label, path in true_negatives:
    print("Message ID:", msg_id)
    print("Predicted label:", pred)
    print("True label:", label)
    print("Path:", path)
    print()

True negatives:
Message ID: 11228
Predicted label: ham
True label: ham
Path: houston trip may heading houston next week talk east desk scheduling settlement systems murray neil volume management jeff richter short term california arrive wednesday night fly back friday afternoon john want meet either days work louise kitchen 03 20 2001 06 56 tim belden hou ect ect cc subject happened 100 well go think meant p l worry obviously long day yesterday england america two countries separated common language good day louise louise kitchen hou ect ect cc subject happened 100 100 mentioned 100 price move fronts understand 100 p l thought sounded bit happy price move p l expected louise kitchen 03 20 2001 06 22 tim belden hou ect ect cc subject happened 100

Message ID: 9356
Predicted label: ham
True label: ham
Path: resume vasant agree vince vasant shanbhogue enron enronxgate 05 01 2001 10 00 vince j kaminski hou ect ect cc subject resume vince seems mostly equity fixed income background credit e

''